# Basics of Interpretation

> Now that we have representations, what do we do with them?

In [ ]:
# |hide
import typing
from collections import UserDict

import numpy as np
import numpy.typing as npt
from numpy.fft import fft, ifft


class HRR(np.ndarray):
    """Thin wrapper around `np.ndarray` for Holographic Reduced Representations
    (HRRs).
    """

    # Incantation needed for subclassing `np.ndarray`.
    def __new__(cls, input_array) -> object:
        obj = np.asarray(input_array).view(cls)
        return obj

    # Same as above
    def __array_finalize__(self, obj: object) -> None:
        if obj is None:
            return

    # The binding operation.
    def bind(self, other: typing.Union[np.ndarray, "HRR"]) -> "HRR":
        """Performcircular convolution.

        Args:
            other (np.ndarray): Second argument.

        Returns:
            Circular convolution of the vector and the other.
        """
        v = ifft(fft(self) * fft(other)).real.view(HRR)
        v /= np.linalg.norm(v)
        return v

    def inverse(self) -> "HRR":
        """Invert the vector for unbinding."""
        return self[np.r_[0, self.size - 1 : 0 : -1]].view(HRR)

    def cosine_similarity(self, other: typing.Union[np.ndarray, "HRR"]) -> float:
        norm = np.linalg.norm(self) * np.linalg.norm(other)
        norm = max(norm, 1e-8)
        return float((self.dot(other)) / norm)


def random(
    num_vectors: int,
    dim: int,
    dtype: npt.DTypeLike = float,
    rng=np.random.default_rng(),
) -> "HRR":
    r"""Create matrix of `n` `d`-dimensional HRR vectors, sampled from the normal
    distribution,
    $$
    \mathcal{N}(\mu=1, \sigma^2 = \frac{1}{d}),
    $$

    Args:
        num_vectors int: The number of vector symbols you wish to generate.
        dim int: The dimension of the vector symbols.
        dtype npt.DTypeLike: The `dtype` of the vector generated. Default: ``float``.
        rng: Random number generator.

    Returns:
        A ``(num_vectors, dim)`` matrix of random vector symbols.
    """
    sd = 1.0 / np.sqrt(dim)
    vs = rng.normal(scale=sd, size=(num_vectors, dim)).astype(dtype)
    norms = np.linalg.vector_norm(vs, axis=1, keepdims=True)
    vs /= norms
    return HRR.__new__(cls=HRR, input_array=vs)


class Codebook(UserDict):
    """Thin dictionary wrapper for codebooks."""

    def __init__(self, symbols: list[str], dim: int) -> None:
        super(Codebook, self).__init__()
        self.dim = dim
        for symbol in symbols:
            self.data[symbol] = random(1, dim).squeeze()


def role_filler_pair(
    struct: typing.Union[
        dict[str, str], dict[str, HRR], dict[HRR, str], dict[HRR, HRR]
    ],
    codebook,
) -> HRR:
    """Create a role-filler pair from a dictionary."""
    v = np.zeros(codebook.dim).view(HRR)
    for role, filler in struct.items():
        if isinstance(role, str):
            role = codebook[role]
        if isinstance(filler, str):
            filler = codebook[filler]
        v += role.bind(filler)
    return v

# Overview of this Section

In this section we will broadly discuss how to represent interpretation in 
programming languages encoded high-dimensional vector space. In order to
work up to this point, first we will discuss some problems with the memory
capacity of VSAs: specifically, how we can get around information-loss
as a result of VSA operations by using *associative memories*. Then, we
will discuss other ways of representing syntax. Finally, we will talk about
what is interpretation and program execution by thinking about a way of 
talking about how programs work: [*denotational semantics*](https://en.wikipedia.org/wiki/Denotational_semantics).
We will then apply this knowledge to another toy language, the LET language.

# Associative Memories and Pointers

Recall our previous example of encoding the simple language $\mathcal{L}_\text{fruit}$
in high-dimensional vectors. We found that we can both *encode* the abstract
syntax into the high-dimensional vectors using VSA operations, as well as 
decode them.

But, as we will show, the decoding breaks down with even $1$-deep nesting of
syntax. We say that a composite expression $\phi$ is $n$-deep by counting the
longest length of composite sub-expressions that it contains. Atomic expressions
are $0$-deep.  Ideally, for languages which are compositional, we would like for any
syntactic expression to be able to be arbitrarily deep. So, we could have
a tuple of tuples of tuples of tuples, etc. But, because of the information
loss inherent in VSA operations, we quickly find that without some indirection,
this becomes untenable.

In [ ]:
from dataclasses import dataclass
from abc import ABCMeta


@dataclass
class L(metaclass=ABCMeta):
    """Abstract base class of our language L."""

    pass


@dataclass
class Atomic(L):
    """Abstract base class of atomic elements in the language."""

    name: str


@dataclass
class Tuple(L):
    """Tuples in L."""

    lhs: L
    rhs: L


@dataclass
class Disjunction(L):
    """Disjunctions in L."""

    lhs: L
    rhs: L


dim = 1_000
T = ["atomic", "tuple", "disjunction"]
R = ["tag", "name", "lhs", "rhs"]
A = ["strawberry", "banana", "apple"]
codebook = Codebook(T + R + A, dim=dim)


def encode(expr: L, codebook: Codebook = codebook) -> HRR:
    """Encode a formula in the language $\mathcal{L}_{\text{fruit}}$."""
    if not isinstance(expr, L):
        raise TypeError("Expected a subclass of L", expr)

    if isinstance(expr, Atomic):
        name = expr.name
        return role_filler_pair(
            {
                "tag": "atomic",
                "name": codebook[name],
            },
            codebook=codebook,
        )
    elif isinstance(expr, Tuple):
        lhs = expr.lhs
        rhs = expr.rhs
        return role_filler_pair(
            {
                "tag": "tuple",
                "lhs": encode(lhs, codebook=codebook),
                "rhs": encode(rhs, codebook=codebook),
            },
            codebook=codebook,
        )
    elif isinstance(expr, Disjunction):
        lhs = expr.lhs
        rhs = expr.rhs
        return role_filler_pair(
            {
                "tag": "disjunction",
                "lhs": encode(lhs, codebook=codebook),
                "rhs": encode(rhs, codebook=codebook),
            },
            codebook=codebook,
        )


def decode(enc: HRR, codebook=codebook, theta: float = 0.2) -> L:
    """Decode a representation back to ``L``."""
    tag = enc.bind(codebook["tag"].inverse())
    t_atom = codebook["atomic"]
    t_tuple = codebook["tuple"]
    t_disj = codebook["disjunction"]

    if tag.cosine_similarity(t_atom) > theta:
        name = enc.bind(codebook["name"].inverse())

        # Recall the name from the codebook
        keys, values = zip(*codebook.items())
        V = np.array(values)
        sims = V @ name
        argmax = np.argmax(sims)

        return Atomic(keys[argmax])
    else:
        # Trick here is that both of the other representations have
        # an `lhs` and a `rhs`.
        lhs = enc.bind(codebook["lhs"].inverse())
        rhs = enc.bind(codebook["rhs"].inverse())
        dec_lhs = decode(lhs, codebook=codebook, theta=theta)
        dec_rhs = decode(rhs, codebook=codebook, theta=theta)
        if tag.cosine_similarity(t_tuple) > theta:
            return Tuple(dec_lhs, dec_rhs)
        elif tag.cosine_similarity(t_disj) > theta:
            return Disjunction(dec_lhs, dec_rhs)
        else:
            raise ValueError("Unkown value!")


# Testing tuples:
straw = Atomic("strawberry")
apple = Atomic("apple")
enc_apple = encode(apple)
tupl = Tuple(straw, apple)
print(f"Original form: {tupl}")
enc_tupl = encode(tupl)
dec_tupl = decode(enc_tupl)
print(f"Decoded form: {dec_tupl}")

print()

# Testing one-deep disjunctions
# disj = Disjunction(tupl, apple)           # Uncommenting this throws errors
# print(f"Original form: {disj}")
# enc_disj = encode(disj)
# dec_disj = decode(enc_disj)
# print(f"Decoded form: {dec_disj}")

Original form: Tuple(lhs=Atomic(name='strawberry'), rhs=Atomic(name='apple'))
Decoded form: Tuple(lhs=Atomic(name='strawberry'), rhs=Atomic(name='apple'))



Even with a simple $1$-deep expression, our decoding function is unable to 
parse out that the left-hand side of `enc_disj` is a tuple. The key problem
here is that we can think of VSA operations as a kind of information compression.
Specifically, binding leads to a loss of information, and this loss
depends on the kind of binding operation that we use (Kelly *et al.*, 2013).

To solve this problem, we need to be able to add (1) indirection into the 
representation, allowing for the information to be preserved even under
many binding operations, (2) a way to "clean-up" or "clarify" noisy representations,
gravitating them back to their original form.

To do this, we use *associative memories*, which are distributed, content-addressable
memories that recontsruct queries based on stored information in their
weights. Associative memories are well-suited for our high-dimensional representation
as they are continuous and differentiable as well.

## Creating a Cleanup and Associative Memory

For more intuition here, we will be creating both a cleanup and associative
memory. A cleanup memory $\mathcal{C}$ is an auto-associative memory which stores $N$
high-dimensional vectors of dimension $D$. If $x$ is a high-dimensional
vector stored in the weights of $\mathcal C$, then $\mathcal C(x) \approx x$. If $x'$ is a 
degraded form of a stored memory item in $C$, then $\mathcal C(x') \approx x$,
where $x$ is the original form of the memory item. Cleanup memories
are useful for recovering the original form of bound variables.

The next associative memory that we will be using is a hetero-associative
memory $\mathcal D$ that stores $N \times N$ items, or $N$ addresses of high-dimensional
vectors and $N$ patterns of high-dimensional vectors. We write to memory
both an address vector and a pattern vector. If $x$ is a high-dimensional
vector stored in the addresses in $\mathcal D$, and $y$ is the a high-dimensional
vector stored in the patterns of $\mathcal D$ associated with $x$
(i.e., the row in both the pattern and address matrix is the same), 
then $\mathcal D(x) \approx y$.  If $x'$ likewise is a degraded form of $x$, 
and $y$ the stored pattern for $x$, then $\mathcal D(x') \approx y$.

In [ ]:
class CleanupMem:
    """Simple clean-up memory."""

    def __init__(self, dim: int, init_capacity: int = 20) -> None:
        # The dimensionality of the data
        self.dim = dim
        # The current capacity of the memory
        self.capacity = init_capacity
        # The number of stored traces
        self.stored_traces = 0
        # The weight matrix
        self.W = np.zeros(shape=(init_capacity, dim))

    def write(self, x: np.ndarray) -> np.ndarray:
        """Write a value to memory."""
        if self.stored_traces >= self.capacity:
            self.W = np.concatenate([self.W, np.zeros((self.capacity, self.dim))])
            self.capacity *= 2
        self.W[self.stored_traces, :] = x
        self.stored_traces += 1
        return x

    def read(self, x: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """Read a value from memory, returning the value and its recalled form"""
        similarities = self.W @ x
        max_sim_idx = np.argmax(np.abs(similarities))
        recalled = self.W[max_sim_idx]
        return x, recalled.view(HRR)

    def __call__(self, x: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        return self.read(x)


class AssocMem:
    def __init__(self, dim: int, init_capacity: int = 20) -> None:
        self.dim = dim
        self.capacity = init_capacity
        self.stored_traces = 0
        # addresses
        self.A = np.zeros((self.capacity, self.dim))
        # patterns
        self.P = np.zeros((self.capacity, self.dim))

    def write(self, x: np.ndarray, y: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """Associate `(x, y)` in memory, returning the values."""
        if self.stored_traces >= self.capacity:
            self.A = np.concatenate([self.A, np.zeros((self.capacity, self.dim))])
            self.P = np.concatenate([self.P, np.zeros((self.capacity, self.dim))])
            self.capacity *= 2
        self.A[self.stored_traces, :] = x
        self.P[self.stored_traces, :] = y
        self.stored_traces += 1
        return x, y.view(HRR)

    def read(self, x: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """Read `x` from memory, returning the `x` and the resulting value."""
        similarities = self.A @ x
        max_sim_idx = np.argmax(np.abs(similarities))
        recalled_pattern = self.P[max_sim_idx]
        return x, recalled_pattern.view(HRR)

    def __call__(self, x: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        return self.read(x)

Let's see how these work. For the cleanup memory, we should be able to degrade
a value and still be able to retrieve the original form.

In [ ]:
dim = 400
X = random(10, dim)
cleanup_mem = CleanupMem(dim=dim)
for x in X:
    cleanup_mem.write(x)

# Test regular recall
x = X[1].squeeze()
_, x_hat = cleanup_mem(x)
print(f"Cosine sim between x and x_hat: {x.cosine_similarity(x_hat)}")

x_degraded = X[1].bind(X[0]).bind(X[2]).bind(X[0].inverse()).bind(X[2].inverse())
_, x_deg_hat = cleanup_mem(x_degraded)
print(f"Cosine sim between x and x_deg_hat: {x.cosine_similarity(x_deg_hat)}")

Cosine sim between x and x_hat: 1.0
Cosine sim between x and x_deg_hat: 1.0


Likewise, we should be able to recall arbitrary associated values even under
degradation.

In [ ]:
dim = 400
A = random(10, dim)
P = random(10, dim)
assoc = AssocMem(dim)
for a, p in zip(A, P):
    assoc.write(a, p)

a = A[1]
p = P[1]
_, p_hat = assoc(a)
print(
    f"Cosine sim between target pattern and recalled pattern: {p.cosine_similarity(p_hat)}"
)

a_degraded = A[1].bind(A[0]).bind(A[2]).bind(A[0].inverse()).bind(A[2].inverse())
_, p_deg_hat = assoc(a_degraded)
print(
    f"Cosine similarity between target pattern and recalled pattern from degraded value: {p_deg_hat.cosine_similarity(p)}"
)

Cosine sim between target pattern and recalled pattern: 1.0000000000000002
Cosine similarity between target pattern and recalled pattern from degraded value: 1.0000000000000002


## Applying memories to encoding and decoding

Now that we have an associative memory and a cleanup memory, how do we apply
it to decoding so that we can preserve information? Recall that role-filler
pair encodings for syntactic forms has some degradation of information,
especially whenever we have multiple binds and superpositions. Therefore,
for each new role-filler pair that we create, what we will do is create
a *reference*. We say that a fresh, random vector $p$ is a reference for some
high-dimensional vector $x$ if $p$ is arbitrary and unrelated with $x$, and
we associate $p$ with $x$ in associative memory.

We also can store the contents of role-filler pairs in cleanup memory. This
helps further with the preservation of information in role-filler pairs. 
Figuring out when and when not to use these methods of indirection 
unfortunately has no real theory behind it. Rather, it is a practical decision
made whenever we notice lots of information loss.

Furthermore, instead of using explicit tag roles, what we will do is 
explicitly superpose the role-filler pair of the contents of the syntax
with the tag itself. This let's us do easier comparison, as well as further
limits the possiblity of information loss.

In [ ]:
def encode_with_references(
    expr: L,
    codebook: Codebook,
    assoc_mem: AssocMem,
    cleanup: CleanupMem,
) -> HRR:
    """Encode an expression with references and indirection."""

    if isinstance(expr, Atomic):
        name = expr.name
        rf = role_filler_pair(
            {
                "name": name,
            },
            codebook=codebook,
        )
        return rf + codebook["atomic"]
    elif isinstance(expr, Tuple):
        lhs = expr.lhs
        rhs = expr.rhs

        enc_lhs = encode_with_references(
            lhs, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup
        )
        enc_rhs = encode_with_references(
            rhs, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup
        )

        cleanup.write(enc_lhs)
        cleanup.write(enc_rhs)

        rfpair = (
            role_filler_pair(
                {"lhs": enc_lhs, "rhs": enc_rhs},
                codebook=codebook,
            )
            + codebook["tuple"]
        )
        ptr = random(1, codebook.dim).squeeze()
        assoc_mem.write(ptr, rfpair)
        return ptr
    elif isinstance(expr, Disjunction):
        lhs = expr.lhs
        rhs = expr.rhs

        enc_lhs = encode_with_references(
            lhs, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup
        )
        enc_rhs = encode_with_references(
            rhs, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup
        )

        cleanup.write(enc_lhs)
        cleanup.write(enc_rhs)

        rfpair = (
            role_filler_pair(
                {"lhs": enc_lhs, "rhs": enc_rhs},
                codebook=codebook,
            )
            + codebook["disjunction"]
        )
        ptr = random(1, codebook.dim).squeeze()
        assoc_mem.write(ptr, rfpair)
        return ptr


dim = 1_000
T = ["atomic", "tuple", "disjunction"]
R = ["tag", "name", "lhs", "rhs"]
A = ["strawberry", "banana", "apple"]
codebook = Codebook(T + R + A, dim=dim)

cleanup_mem = CleanupMem(dim=codebook.dim)
for item in codebook.values():
    cleanup_mem.write(item)
assoc_mem = AssocMem(dim=dim)

straw = Atomic("strawberry")
enc_straw = encode_with_references(
    straw, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup_mem
)
print(
    f"Is the encoded `straw` atomic?: {enc_straw.cosine_similarity(codebook['atomic'])}"
)



Is the encoded `straw` atomic?: 0.7218637832801331


In [ ]:
# Testing tuples
banana = Atomic("banana")
enc_banana = encode_with_references(
    banana, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup_mem
)

t = Tuple(straw, banana)
ptr_t = encode_with_references(
    t, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup_mem
)

print(f"Is the pointer a tuple?: {ptr_t.cosine_similarity(codebook['tuple'])}")

_, deref_t = assoc_mem.read(ptr_t)

print(f"Is the derefenced tuple a tuple?: {deref_t.cosine_similarity(codebook['tuple'])}")
lhs = deref_t.bind(codebook["lhs"].inverse())
_, lhs = cleanup_mem.read(lhs)
print(f"Similarity between lhs and atomic: {lhs.cosine_similarity(codebook['atomic'])}")
print(f"Is the lhs a strawberry?: {lhs.bind(codebook['name'].inverse()).cosine_similarity(codebook['strawberry'])}")

Is the pointer a tuple?: 0.007466406248505364
Is the derefenced tuple a tuple?: 0.5889989642948561
Similarity between lhs and atomic: 0.7218637832801331
Is the lhs a strawberry?: 0.566608025894019


#### Decoding

Now that we've preserved the information more using indirection, we can also
have an easier time in decoding compound representations.

In [ ]:
t = Tuple(straw, banana)
t_nest = Tuple(t, t)
ptr_t = encode_with_references(
    t_nest, codebook=codebook, assoc_mem=assoc_mem, cleanup=cleanup_mem
)

_, deref_t = assoc_mem.read(ptr_t)
_, lhs_ptr = cleanup_mem.read(deref_t.bind(codebook["lhs"].inverse()))
_, deref_lhs = assoc_mem.read(lhs_ptr)
print(f'Is the left-hand side a tuple?: {deref_lhs.cosine_similarity(codebook["tuple"])}')

Is the left-hand side a tuple?: 0.5889989642948561


Implementing decoding the forms with indirection is left as an exercise for
the reader. It is always important to remember: you can not be sure that a value
that you have is an atomic value that isn't the result of some kind of indirection.
Therefore, one must always be wary of testing for whether the value is atomic.
To do so, one can simply test whether some high-dimensional encoded vector
is sufficiently similar to the `atomic` tag. Otherwise, you can treat it as a
reference, and dereference it from memory.

In [ ]:
def decode_with_references(
    enc: HRR,
    codebook: Codebook,
    assoc_mem: AssocMem,
    cleanup_mem: CleanupMem,
) -> L:
    """Decode from our novel encoding with references."""
    ...

# Alternative Representations of Syntax

## Trees

## Lists

# Denotational Semantics

# The LET Programming Language

The LET Programming language is a toy language that contains: simple
expressions like addition and subtraction, testing whether or not an item
is $0$, function application, conditional evaluation, and variable binding
using `let` expressions.

Let $\mathcal{L}_{\text{let}}$ be the language defined by the following BNF
specification:
```bnf
<program> ::= <expression>

<expression> ::= <number>
              |  -(<expression>, <expression>)
              | zero? (<expression>)
              | if <expression> then <expression> else <expression>
              | <identifier>
              | let <expression> = <expression> in <expression>
```

# Denotational Semantics

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()